# Acquisition section.
## Here we will acquire crawled data from web and from excel, then place it into data frame

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xlrd
import math

### First we obtain the table of countries

In [2]:
#Crawling the CO2 emission
url = 'https://www.worldometers.info/co2-emissions/co2-emissions-by-country/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
CO2FossilByCountry = soup.find("table", attrs = {"id" : "example2"})
tableRows = CO2FossilByCountry.find("tbody")

### After we obtained the table of countries, we will get the href links to each country.
### Then, we will get each country's data, while summing the data into a list

In [3]:
#First loop the get all country's tables
year, FossilCO2, hrefLinks, country, avgCO2Fossil = (list() for i in range(5))
isContinue, isFirstLoop = (True for i in range(2))
i = 0
for row in tableRows.findAll("tr"):
    row.find("td", attrs = {"style":"font-weight: bold; font-size:15px; text-align:left; padding-top:10px; padding-bottom:10px"})
    hrefLinks.append(row.find("a").get("href"))
    country.append(row.find("a").text)
#Second loop the get each coyntry's data from its table
for links in hrefLinks:
    prefix = "https://www.worldometers.info"
    suffix = links
    currUrl = prefix+suffix
    currResponse = requests.get(currUrl)
    currSoup = BeautifulSoup(currResponse.content, 'html.parser')
    tbl = currSoup("table", attrs={"class":"table table-striped table-bordered table-hover table-condensed table-list"})[0]
    for row in tbl ("tr"):
        cells=row("td")
        if (len(cells)<7): #Catching the cells of the table itself
            continue
        if isFirstLoop:
            avgCO2Fossil.append(int(cells[1].get_text().strip().replace(',', '')))
        else:
            avgCO2Fossil[i] = int(avgCO2Fossil[i] + (int(cells[1].get_text().strip().replace(',', ''))))
            i = i+1
        if isContinue:
             year.append(cells[0].get_text().strip())
    isContinue = False
    isFirstLoop = False
    i = 0

### At this point we got a sum of each country's data by year, now we will make an average for the list by the years

In [4]:
globalAvgCO2Fossil = avgCO2Fossil.copy()
for i in range(0,len(globalAvgCO2Fossil)):
    globalAvgCO2Fossil[i] = int(math.floor(globalAvgCO2Fossil[i] / len(country)))

### Eventually, creating the data frame

In [5]:
data = {"Year": year, "FossilCO2":globalAvgCO2Fossil}
df_GlobalCO2Fossil = pd.DataFrame(data=data)
df_GlobalCO2Fossil.to_csv("CO2Fossil.csv")

### Here we acquire the data from excel file and creating CSV file

In [6]:
#Loading the excel file pre-downloaded, taking the row "Total World" for each column <Year> then transposing it.
Year = list()
NG = list()
wb = xlrd.open_workbook('Natural gas demand.xlsx')
sh = wb.sheet_by_name('Table 9.6')
for col in sh.row_values(2):
    if type(col) != str:
        Year.append(int(float(col)))
for col in sh.row_values(78):
    if type(col) != str:
        NG.append((int(col)))
data = {'Year':Year, 'Natural Gas Demand':NG}
df_NaturalGasDemand = pd.DataFrame(data=data)
df_NaturalGasDemand.to_csv("NaturalGasDemand.csv")